# Fig09

In [ ]:
from data_import import *
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from plotting import *

device = 'cuda:0'

df_orig = load_result_table('feeds/phase_diagram_T32').reset_index()
df_orig = df_orig[df_orig['epoch']==499]
figure_dir = FIGURE_DIR

In [ ]:
allowed_p = [2,4,8,16,32,64,128]
allowed_model_dim = [2,4,8,16,32,64,128]

df_orig = df_orig[df_orig['p'].isin(allowed_p) & df_orig['model_dim'].isin(allowed_model_dim)]

In [ ]:
def plot_output(config,ax):
  model, embd = load_model(config)
  colors = list(plt.cm.tab20b.colors)

  weights = model.fc2.weight.detach().cpu().numpy()
  bias = model.fc2.bias.detach().cpu().numpy()
  for token in range(32):
    if token == 22:
      pass
    if 'BOS' in config.dataset_type:
      idx = 1
      X = np.zeros((10,11))
      for i in range(1,10):
          X[i] = token
          X[i,1 : i+1] = 22  
      X[:,0] = 32 
    else:
      idx = 0
      X = np.zeros((10,10))
      for i in range(1,10):
          X[i] = token
          X[i,: i] = 22   
    X = torch.tensor(X,dtype=torch.long).to(device)[1:]
    model.forward(X)
    hidden_p = model.activ(model.b)[:,idx].detach().cpu().numpy()
    for i in range(1,weights.shape[0]-1):
      ys = []
      for x in range(9):
        y = (hidden_p[x] @ weights[i]) + bias[i]
        ys.append(y)
        
      ax.plot(range(1,10),ys,c=colors[i],alpha=0.1 if token != 31 else 1.0)
  for i in range(9):
    ax.plot([],[],c=colors[i],label=f'{i+1}')



In [ ]:
df = df_orig.sort_values('val_acc',ascending=False).groupby(CONFIG_COLS).head(1).reset_index()
allowed_model_dim_ = allowed_model_dim[::-1]
for name, model_config in MODELS.items():
    print(model_config)
    
    a = df[(df.attention_input == model_config['attention_input'])
            & (df.no_softmax == model_config['no_softmax']) 
            & (df.dataset_type == model_config['dataset'])]
    
    

    fig, axs = plt.subplots(len(allowed_p),len(allowed_model_dim_),figsize=(20,20),sharex=True)
    for p in allowed_p:
        # set ylabel
        axs[-1,allowed_p.index(p)].set_xlabel(f'p={p}')
    for model_dim in allowed_model_dim_:
        # set xlabel
        axs[allowed_model_dim_.index(model_dim),0].set_ylabel(f'model_dim={model_dim}')
    
    for p in allowed_p:
        for model_dim in allowed_model_dim_:
            
            ax = axs[allowed_model_dim_.index(model_dim),allowed_p.index(p)]
            config = a[(a['p']==p) & (a['model_dim']==model_dim) ].set_index('name').iloc[0]
            plot_output(config,ax)
        
            ax.set_title(f'{config.val_acc:.2f}%')

    plt.savefig(FIGURE_DIR / f'outputs_{name}.png',dpi=300,bbox_inches='tight') 
    plt.show()